## Focaccia

- autolyse 20 min
- bulk fermentation 2 hours, fold once
- final fermentation 1-1.5 hours

In [1]:
import pandas as pd
grams_per_ounce = 28.3495

In [2]:
num_loaves = 4
num_pounds = 0
num_ounces = 0
num_grams = 1170

In [3]:
loaf_weight = (num_pounds * 16 + num_ounces) * grams_per_ounce + num_grams
total_weight = (num_loaves * loaf_weight)
print ("{:,.0f} loaves at {:,.0f} grams".
       format(num_loaves, loaf_weight, num_pounds, num_ounces))
print ("total weight: {:,.2f} grams".format(total_weight,total_weight * grams_per_ounce))

4 loaves at 1,170 grams
total weight: 4,680.00 grams


In [4]:
formula = pd.DataFrame.from_dict({
    "ap-flour": 100, 
    "water": 72.7, 
    "yeast": 0.76,
    "salt": 2.2,
    "olive oil": 13.6,
    "malt": 0.5
}, orient='index', columns=["baker%"])
print ("overall formula")
formula_total = formula['baker%'].sum()
print("overall formula total = %s%%" % formula_total)
formula

overall formula
overall formula total = 189.75999999999996%


,baker%
ap-flour,100.00
water,72.70
yeast,0.76
salt,2.20
olive oil,13.60
malt,0.50


In [5]:
pre_ferment = .45
poolish = pd.DataFrame.from_dict({
    "ap-flour": 100,
    "water": 100, 
    "yeast": 0.1
}, orient='index', columns=["baker%"])
print ("poolish")
poolish_total = poolish['baker%'].sum()
print("poolish total = %s%%" % poolish_total)
poolish

poolish
poolish total = 200.1%


,baker%
ap-flour,100.0
water,100.0
yeast,0.1


In [6]:
poolish_weight = total_weight * 100 / formula_total * pre_ferment * poolish_total/poolish.at['ap-flour','baker%']
print ("poolish weight = {:,.0f} grams".format(poolish_weight))
poolish['grams'] = poolish.apply(lambda row: '{:.0f}'.format(row['baker%']*poolish_weight/poolish_total), axis=1)
poolish['oz'] = poolish.apply(lambda row: '{:.1f}'.format(float(row['grams']) / grams_per_ounce), axis=1)
poolish = poolish[['baker%', 'grams', 'oz']]
poolish

poolish weight = 2,221 grams


,baker%,grams,oz
ap-flour,100.0,1110,39.2
water,100.0,1110,39.2
yeast,0.1,1,0.0


In [7]:
final_dough = formula
final_dough['grams'] = final_dough.apply(lambda row: '{:.0f}'.format(row['baker%']*total_weight / formula_total), axis=1)
final_dough.at['poolish','grams'] = '{:.0f}'.format(poolish_weight)
final_dough.at['poolish','baker%'] = '{:.0f}'.format(poolish_weight / float(final_dough.at['ap-flour','grams']) * 100)
for index, row in poolish.iterrows():
    final_dough.at[index,'grams'] = float(final_dough.at[index,'grams']) - float(row['grams'])
final_dough['oz'] = final_dough.apply(lambda row: '{:,.1f}'.format(float(row['grams'])/grams_per_ounce), axis=1)
final_dough = final_dough[['baker%', 'grams', 'oz']]
print("final dough")
final_dough

final dough


/var/folders/hs/f1lmyqbj5m780x8vmm94m6v80000gn/T/ipykernel_22378/1379030967.py:4: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '90' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  final_dough.at['poolish','baker%'] = '{:.0f}'.format(poolish_weight / float(final_dough.at['ap-flour','grams']) * 100)


,baker%,grams,oz
ap-flour,100.0,1356.0,47.8
water,72.7,683.0,24.1
yeast,0.76,18.0,0.6
salt,2.2,54,1.9
olive oil,13.6,335,11.8
malt,0.5,12,0.4
poolish,90,2221,78.3
